<a href="https://colab.research.google.com/github/WonJunPark/Resnext-Xception-Ensemble-Inference-code-review/blob/master/Resnext-Xception-Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resnext & Xception Ensemble (Inference)


This kernel outputs the ensemble of the results from https://www.kaggle.com/

1.   khoongweihao/frames-per-video-viz and https://www.kaggle.com/greatgamedota/xception-binary-classifier-inference (not original, modified learning rate and epochs)
2.   Frames per video at 64 (best found)




# 데이터 불러오기

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My Drive/deepfake-detection-challenge

/content/gdrive/My Drive/deepfake-detection-challenge


In [0]:
test_dir = "/content/gdrive/My Drive/deepfake-detection-challenge/test_videos/"
test_videos = sorted([x for x in os.listdir(test_dir) if x[-4:] == ".mp4"])

In [0]:
frame_h = 5
frame_l = 5
len(test_videos)

400

# 버전 확인

In [0]:
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

PyTorch version: 1.4.0
CUDA version: 10.1
cuDNN version: 7603


In [0]:
gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
gpu

device(type='cpu')

# Resnext model

In [0]:
import sys
sys.path.insert(0, "/kaggle/input/blazeface-pytorch")
sys.path.insert(0, "/kaggle/input/deepfakes-inference-demo")

Blazeface
- google reserach의 빠르고 가벼운 얼굴 탐지기
- 눈2 귀2 코1 입1에 대한 6개의 키포인트를 예측
- https://github.com/hollance/BlazeFace-PyTorch

In [0]:
from blazeface import BlazeFace
facedet = BlazeFace().to(gpu)
facedet.load_weights("/kaggle/input/blazeface-pytorch/blazeface.pth") # blazeface.pth : 훈련 된 모델의 가중치
facedet.load_anchors("/kaggle/input/blazeface-pytorch/anchors.npy") # anchors.npy : 앵커 박스가있는 룩업 테이블
_ = facedet.train(False)

VideoReader
- 파이썬에서 비디오 파일 읽기를 단순화
- https://github.com/postpop/videoreader

FaceExtractor
- 얼굴 추출기는 비디오에서 얼굴 데이터 세트를 만드는 도구입니다. 입력 데이터 (사진 또는 비디오)를 읽고 그 안에있는 얼굴을 검색하고 얼굴을 찾으면 자르고 원본 입력 파일을 수정하지 않고 출력 폴더에 인쇄합니다.
- https://github.com/Baycosinus/face-extractor

In [0]:
from helpers.read_video_1 import VideoReader
from helpers.face_extract_1 import FaceExtractor

frames_per_video = 64 #frame_h * frame_l
video_reader = VideoReader() # 비디오를 읽음
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video) #비디오에서 얼굴 추출
face_extractor = FaceExtractor(video_read_fn, facedet)

In [0]:
input_size = 224

In [0]:
from torchvision.transforms import Normalize #정규화

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
normalize_transform = Normalize(mean, std)

In [0]:
def isotropically_resize_image(img, size, resample=cv2.INTER_AREA):
    h, w = img.shape[:2]
    if w > h:
        h = h * size // w
        w = size
    else:
        w = w * size // h
        h = size

    resized = cv2.resize(img, (w, h), interpolation=resample)
    return resized

def make_square_image(img):
    h, w = img.shape[:2]
    size = max(h, w)
    t = 0
    b = size - h
    l = 0
    r = size - w
    return cv2.copyMakeBorder(img, t, b, l, r, cv2.BORDER_CONSTANT, value=0)

In [0]:
import torch.nn as nn
import torchvision.models as models #resnet model 불러오기

class MyResNeXt(models.resnet.ResNet):
    def __init__(self, training=True):
        super(MyResNeXt, self).__init__(block=models.resnet.Bottleneck,
                                        layers=[3, 4, 6, 3], 
                                        groups=32, 
                                        width_per_group=4)
        self.fc = nn.Linear(2048, 1)

In [0]:
checkpoint = torch.load("/kaggle/input/deepfakes-inference-demo/resnext.pth", map_location=gpu) # 사전학습 모델

model = MyResNeXt().to(gpu)
model.load_state_dict(checkpoint) #학습해둔 모델을 불러옴
_ = model.eval()

del checkpoint

In [0]:
def predict_on_video(video_path, batch_size):
    try:

        # Find the faces for N frames in the video.
        faces = face_extractor.process_video(video_path) # 비디오에서 얼굴을 추출

        # Only look at one face per frame.
        face_extractor.keep_only_best_face(faces) #한개의 얼굴에 프레임을 씌움
        
        if len(faces) > 0:
            # NOTE: When running on the CPU, the batch size must be fixed
            # or else memory usage will blow up. (Bug in PyTorch?)
            x = np.zeros((batch_size, input_size, input_size, 3), dtype=np.uint8) #one-hot 인코딩

            n = 0
            for frame_data in faces: # 1개의 mp4 파일
                for face in frame_data["faces"]: # mp4파일 내의 얼굴 이미지들
                    # Resize to the model's required input size.
                    # We keep the aspect ratio intact and add zero
                    # padding if necessary.                    
                    resized_face = isotropically_resize_image(face, input_size) # input_size에 맞게 이미지 전처리 (input_size = 224)
                    resized_face = make_square_image(resized_face)  # 얼굴 부분에 squear을 만듬

                    if n < batch_size:
                        x[n] = resized_face
                        n += 1
                    else:
                        print("WARNING: have %d faces but batch size is %d" % (n, batch_size))
                    
                    # Test time augmentation: horizontal flips.
                    # TODO: not sure yet if this helps or not
                    #x[n] = cv2.flip(resized_face, 1)
                    #n += 1

            if n > 0: #전처리 완료된 이미지들을 모델에 대입
                x = torch.tensor(x, device=gpu).float()

                # Preprocess the images.
                x = x.permute((0, 3, 1, 2))

                for i in range(len(x)):
                    x[i] = normalize_transform(x[i] / 255.) #정규화

                # Make a prediction, then take the average.
                with torch.no_grad():
                    y_pred = model(x) # resnext 모델로 예측
                    y_pred = torch.sigmoid(y_pred.squeeze())
                    return y_pred[:n].mean().item()

    except Exception as e:
        print("Prediction error on video %s: %s" % (video_path, str(e)))

    return 0.5

concurrent.futures 
- 병렬 작업 실행
- https://docs.python.org/ko/3.7/library/concurrent.futures.html

In [0]:
from concurrent.futures import ThreadPoolExecutor

def predict_on_video_set(videos, num_workers):
    def process_file(i):
        filename = videos[i]
        y_pred = predict_on_video(os.path.join(test_dir, filename), batch_size=frames_per_video) #모든 test set에 대한 예측값
        return y_pred

    with ThreadPoolExecutor(max_workers=num_workers) as ex:
        predictions = ex.map(process_file, range(len(videos))) #병렬 작업

    return list(predictions)

In [0]:
speed_test = False  #속도를 확인하려면 True값으로 지정 # you have to enable this manually

In [0]:
if speed_test:
    start_time = time.time()
    speedtest_videos = test_videos[:5]
    predictions = predict_on_video_set(speedtest_videos, num_workers=4)
    elapsed = time.time() - start_time
    print("Elapsed %f sec. Average per video: %f sec." % (elapsed, elapsed / len(speedtest_videos)))

In [0]:
predictions = predict_on_video_set(test_videos, num_workers=4) #병렬 처리 시작

In [0]:
submission_df_resnext = pd.DataFrame({"filename": test_videos, "label": predictions})
submission_df_resnext.to_csv("submission_resnext.csv", index=False)

# Xception model

In [1]:
# 설치
!pip install ../input/deepfake-xception-trained-model/pytorchcv-0.0.55-py2.py3-none-any.whl --quiet

ERROR: Could not install packages due to an EnvironmentError: [Errno 2] No such file or directory: '/input/deepfake-xception-trained-model/pytorchcv-0.0.55-py2.py3-none-any.whl'



In [0]:
# 데이터 불러오기, resnext와 동일
test_dir = "/kaggle/input/deepfake-detection-challenge/test_videos/"

test_videos = sorted([x for x in os.listdir(test_dir) if x[-4:] == ".mp4"])
len(test_videos)

In [0]:
# resnext와 동일
gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
# resnext와 동일
import sys
sys.path.insert(0, "/kaggle/input/blazeface-pytorch")
sys.path.insert(0, "/kaggle/input/deepfakes-inference-demo")

In [0]:
# 영상 전처리, resnext와 동일
from blazeface import BlazeFace
facedet = BlazeFace().to(gpu)
facedet.load_weights("/kaggle/input/blazeface-pytorch/blazeface.pth")
facedet.load_anchors("/kaggle/input/blazeface-pytorch/anchors.npy")
_ = facedet.train(False)

In [0]:
# 영상 전처리, resnext와 동일
from helpers.read_video_1 import VideoReader
from helpers.face_extract_1 import FaceExtractor

frames_per_video = 64 # originally 4

video_reader = VideoReader()
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn, facedet)

In [0]:
# resnext에서는 input_size를 224로 줬는데, xception net은 150으로 줌 (?)
input_size = 150

In [0]:
# 정규화, resnext와 동일
from torchvision.transforms import Normalize

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
normalize_transform = Normalize(mean, std)

In [0]:
# Face extraction, resnext와 동일
def isotropically_resize_image(img, size, resample=cv2.INTER_AREA):
    h, w = img.shape[:2]
    if w > h:
        h = h * size // w
        w = size
    else:
        w = w * size // h
        h = size

    resized = cv2.resize(img, (w, h), interpolation=resample)
    return resized


def make_square_image(img):
    h, w = img.shape[:2]
    size = max(h, w)
    t = 0
    b = size - h
    l = 0
    r = size - w
    return cv2.copyMakeBorder(img, t, b, l, r, cv2.BORDER_CONSTANT, value=0)

In [0]:
ls ../input/deepfake-xception-trained-model

In [0]:
# ★ 제일 중요한 부분인데, 토치 공부가 미흡해서 해석이 힘듬..
from pytorchcv.model_provider import get_model
model = get_model("xception", pretrained=False)
model = nn.Sequential(*list(model.children())[:-1]) # Remove original output layer

class Pooling(nn.Module):
  def __init__(self):
    super(Pooling, self).__init__()
    
    self.p1 = nn.AdaptiveAvgPool2d((1,1))
    self.p2 = nn.AdaptiveMaxPool2d((1,1))

  def forward(self, x):
    x1 = self.p1(x)
    x2 = self.p2(x)
    return (x1+x2) * 0.5

model[0].final_block.pool = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)))

class Head(torch.nn.Module):
  def __init__(self, in_f, out_f):
    super(Head, self).__init__()
    
    self.f = nn.Flatten()
    self.l = nn.Linear(in_f, 512)
    self.d = nn.Dropout(0.5)
    self.o = nn.Linear(512, out_f)
    self.b1 = nn.BatchNorm1d(in_f)
    self.b2 = nn.BatchNorm1d(512)
    self.r = nn.ReLU()

  def forward(self, x):
    x = self.f(x)
    x = self.b1(x)
    x = self.d(x)

    x = self.l(x)
    x = self.r(x)
    x = self.b2(x)
    x = self.d(x)

    out = self.o(x)
    return out

class FCN(torch.nn.Module):
  def __init__(self, base, in_f):
    super(FCN, self).__init__()
    self.base = base
    self.h1 = Head(in_f, 1)
  
  def forward(self, x):
    x = self.base(x)
    return self.h1(x)

net = []
model = FCN(model, 2048)
model = model.cuda()
model.load_state_dict(torch.load('../input/deepfake-xception-trained-model/model.pth')) # new, updated
net.append(model)

In [0]:
# 모델 예측, resnext와 동일
def predict_on_video(video_path, batch_size):
    try:
        # Find the faces for N frames in the video.
        faces = face_extractor.process_video(video_path)

        # Only look at one face per frame.
        face_extractor.keep_only_best_face(faces)
        
        if len(faces) > 0:
            # NOTE: When running on the CPU, the batch size must be fixed
            # or else memory usage will blow up. (Bug in PyTorch?)
            x = np.zeros((batch_size, input_size, input_size, 3), dtype=np.uint8)

            # If we found any faces, prepare them for the model.
            n = 0
            for frame_data in faces:
                for face in frame_data["faces"]:
                    # Resize to the model's required input size.
                    # We keep the aspect ratio intact and add zero
                    # padding if necessary.                    
                    resized_face = isotropically_resize_image(face, input_size)
                    resized_face = make_square_image(resized_face)

                    if n < batch_size:
                        x[n] = resized_face
                        n += 1
                    else:
                        print("WARNING: have %d faces but batch size is %d" % (n, batch_size))
                    
                    # Test time augmentation: horizontal flips.
                    # TODO: not sure yet if this helps or not
                    #x[n] = cv2.flip(resized_face, 1)
                    #n += 1

            if n > 0:
                x = torch.tensor(x, device=gpu).float()

                # Preprocess the images.
                x = x.permute((0, 3, 1, 2))

                for i in range(len(x)):
                    x[i] = normalize_transform(x[i] / 255.)
                    # x[i] = x[i] / 255.

                # Make a prediction, then take the average.
                with torch.no_grad():
                    y_pred = model(x)
                    y_pred = torch.sigmoid(y_pred.squeeze())
                    return y_pred[:n].mean().item()

    except Exception as e:
        print("Prediction error on video %s: %s" % (video_path, str(e)))

    return 0.5

In [0]:
# 병렬 처리, resnext와 동일
from concurrent.futures import ThreadPoolExecutor

def predict_on_video_set(videos, num_workers):
    def process_file(i):
        filename = videos[i]
        y_pred = predict_on_video(os.path.join(test_dir, filename), batch_size=frames_per_video)
        return y_pred

    with ThreadPoolExecutor(max_workers=num_workers) as ex:
        predictions = ex.map(process_file, range(len(videos)))

    return list(predictions)

In [0]:
# 속도 확인, resnext와 동일
speed_test = False

if speed_test:
    start_time = time.time()
    speedtest_videos = test_videos[:5]
    predictions = predict_on_video_set(speedtest_videos, num_workers=4)
    elapsed = time.time() - start_time
    print("Elapsed %f sec. Average per video: %f sec." % (elapsed, elapsed / len(speedtest_videos)))

In [0]:
%%time
model.eval()
predictions = predict_on_video_set(test_videos, num_workers=4) #전체 mp4 파일 예측 시간 확인

In [0]:
# xception 모델 예측
submission_df_xception = pd.DataFrame({"filename": test_videos, "label": predictions})
submission_df_xception.to_csv("submission_xception.csv", index=False)

# Ensemble of Resnext and Xception

In [0]:
# resnext 결과
submission_df_resnext.head()

In [0]:
# xception 결과
submission_df_xception.head()

In [0]:
submission_df = pd.DataFrame({"filename": test_videos})

In [0]:
r1 = 0.46441 #xception 비율
r2 = 0.52189 # resnenext 비율
total = r1 + r2
r11 = r1/total
r22 = r2/total

In [0]:
submission_df["label"] = r22*submission_df_resnext["label"] + r11*submission_df_xception["label"] #앙상블

In [0]:
submission_df.to_csv("submission.csv", index=False)